[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2023 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

## Task 10.2 Systemdiskussion für FIR-Filter

64BE42BAEF

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from scipy.signal import tf2zpk, dlti
from scipy.signal import TransferFunction, ZerosPolesGain, bilinear
from scipy.signal import dimpulse, dstep, dfreqresp, dbode
from scipy.fftpack import fft
import sys
import os

In [ ]:
cur_fol = os.getcwd()
print(cur_fol)
sys.path.append(cur_fol + '/../')  # access to root folder

In [ ]:
from sig_sys_tools import plot_zplane  # sig_sys_tools.py in the root folder

In [ ]:
Nf = 2**10  # values along frequency axis
Om = np.arange(Nf) * (2*np.pi/Nf)  # get frequency vector for bode, fft

# Define FIR System 

In [ ]:
# FIR...finite impulse response, i.e. we actually refer to a non-recursive system
# linear-phase type I, i.e. odd length of b, axial symmetry
b = np.array([1, 0, 1, 0, 17/4, 0, 17/4, 0, 1, 0, 1])
a = np.zeros_like(b)  # length(a)==length(b)!
a[0] = 1
Nt = 2**4  # values along time axis
fir_flag = True

# System Descriptions

In [ ]:
z, p, k = tf2zpk(b, a)

# possible descriptions from scipy.signal package
sys1 = dlti(b, a)
sys2 = TransferFunction(b, a, dt=True)
sys3 = dlti(z, p, k)
sys4 = ZerosPolesGain(z, p, k, dt=True)

sys = sys1  # use one of the above possibilities

# Impulse Response

In [ ]:
h = dimpulse(sys, n=Nt)
plt.stem(np.squeeze(h[0]), np.squeeze(h[1]),
         linefmt='C0:', markerfmt='C0o', basefmt='C0:')
plt.xlabel(r'$k$')
plt.ylabel(r'$h[k]$')
plt.grid(True)

# Step Response

In [ ]:
he = dstep(sys, n=Nt)
plt.stem(np.squeeze(he[0]), np.squeeze(he[1]),
         linefmt='C0:', markerfmt='C0o', basefmt='C0:')
plt.xlabel(r'$k$')
plt.ylabel(r'$h_\epsilon[k]$')
plt.grid(True)

# Pole / Zero Map

In [ ]:
plot_zplane(z, p, k)

# Bode Plot

In [ ]:
_, level, phase = dbode(sys, Om)
_, H = dfreqresp(sys, Om)

Hmax = 30
Hmin = Hmax - 50

if fir_flag:
    # for FIR filters we can also utilize the FFT of the b-coefficients as
    Hfft = fft(b, Nf)  # zeropadded FFT is interpolation towards DTFT

plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(Om, level, 'C2')
plt.plot(Om, 20*np.log10(np.abs(H)), 'C1')
if fir_flag:
    plt.plot(Om, 20*np.log10(np.abs(Hfft)), 'C0')
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
plt.ylim(Hmin, Hmax)
#plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'level / dB')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(Om, phase, 'C2')
plt.plot(Om, np.unwrap(np.angle(H))*180/np.pi, 'C1')
if fir_flag:
    plt.plot(Om, np.unwrap(np.angle(Hfft))*180/np.pi, 'C0')
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
plt.yticks(np.arange(-180*10, 0+180, 180))
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'phase / deg')
plt.grid(True)
plt.savefig('fir_filter_64BE42BAEF_bode.pdf')

# Impulse Response, Magnitude/Phase of Zerophase System

\begin{align}
&H_0(z) = z^5 + z^3 + \frac{17}{4}\cdot z^{+1} + \frac{17}{4}\cdot z^{-1} + z^{-3} + z^{-5}
&\\\bullet-\circ\\
&h_0[k] = \delta[k+5] + \delta[k+3] + \frac{17}{4}\cdot\delta[k+1] + \frac{17}{4}\cdot\delta[k-1] + \delta[k-3] + \delta[k-5]
\end{align}

In [ ]:
h = dimpulse(sys, n=Nt)
plt.stem(np.squeeze(h[0])-5, np.squeeze(h[1]),
         linefmt='C0:', markerfmt='C0o', basefmt='C0:')
plt.xlabel(r'$k$')
plt.ylabel(r'$h_0[k]$')
plt.grid(True)

In [ ]:
H01 = 2*np.cos(5*Om)
H02 = 2*np.cos(3*Om)
H03 = 17/2*np.cos(1*Om)
H0 = H01+H02+H03

Hphase = np.zeros_like(H0)
Hphase[H0 <= 0] = 180
Hphase[np.logical_and(H0 >= 0, Om > np.pi)] = 360


plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(Om, H01, 'C0', label=r'$2\cos(5\Omega)$')
plt.plot(Om, H02, 'C1', label=r'$2\cos(3\Omega)$')
plt.plot(Om, H03, 'C2', label=r'$\frac{17}{2}\cos(\Omega)$')
plt.plot(Om, H0, 'C6',
         label=r'$H_0(\Omega)$')
plt.plot(Om, np.abs(H0), 'C3',
         label=r'$|H_0(\Omega)|$', lw=2)
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
#plt.xlabel(r'$\Omega$ / rad')
plt.legend(loc='lower right')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(Om, Hphase, color='C3', lw=2, label=r'$\angle H_0$')
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
plt.yticks([0, 180, 360], [r'$0$', r'$180$', r'$360$'])
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel('degree')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('fir_filter_64BE42BAEF_zerophase.pdf')

In [ ]:
# check polarity inversion
N = 16
kx = np.arange(N)
phi = np.pi
x = np.cos(phi*kx)
xshift = np.cos(phi*(kx+5))  # simulate zero phase LTI system!
y = np.convolve(np.squeeze(h[1]), xshift)
ky = np.arange(y.size)
plt.plot(kx, x, label=r'$x[k]=\cos(\pi k)$')
plt.plot(ky, y/12.5, label=r'$y[k]=x[k] \, *_k \, h_0[k]/12.5$')
plt.legend()
plt.xlabel(r'$x[k]$')
plt.title('polarity inversion between in and out as expected')
plt.grid(True)

# Impulse Response, Magnitude/Phase of Linear Phase System = Pure Delay 

\begin{align}
&H_d(z) = z^{-5}\bullet-\circ h_d[k] = \delta[k-5]
\end{align}

In [ ]:
bd = [0, 0, 0, 0, 0, 1]
ad = [1, 0, 0, 0, 0, 0]
sysd = dlti(bd, ad)

In [ ]:
hd = dimpulse(sysd, n=Nt)
plt.stem(np.squeeze(hd[0]), np.squeeze(hd[1]),
         linefmt='C0:', markerfmt='C0o', basefmt='C0:')
plt.xlabel(r'$k$')
plt.ylabel(r'$h_d[k]$')
plt.grid(True)

In [ ]:
_, level, phase = dbode(sysd, Om)
_, H = dfreqresp(sysd, Om)

Hmax = 10
Hmin = Hmax - 20

if fir_flag:
    # for FIR filters we can also utilize the FFT of the b-coefficients as
    Hfft = fft(bd, Nf)  # zeropadded FFT is interpolation towards DTFT

plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(Om, level, 'C2')
plt.plot(Om, 20*np.log10(np.abs(H)), 'C1')
if fir_flag:
    plt.plot(Om, 20*np.log10(np.abs(Hfft)), 'C0')
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
plt.ylim(Hmin, Hmax)
#plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'level / dB')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(Om, phase, 'C2')
plt.plot(Om, np.unwrap(np.angle(H))*180/np.pi, 'C1')
if fir_flag:
    plt.plot(Om, np.unwrap(np.angle(Hfft))*180/np.pi, 'C0')
plt.xlim(0, 2*np.pi)
plt.xticks(np.arange(9)*2*np.pi/8,
           [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$',
            r'$\pi$', r'$\frac{5\pi}{4}$', r'$\frac{3\pi}{2}$',
            r'$\frac{7\pi}{4}$', r'$2\pi$'])
plt.yticks(np.arange(-180*10, 0+180, 180))
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'phase / deg')
plt.grid(True)
plt.savefig('fir_filter_64BE42BAEF_bode_delay.pdf')

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.